# 라이브러리

In [99]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt

from math import sqrt
from tqdm import tqdm
from konlpy.tag import *
from datetime import timedelta

from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [25]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

# 종목명 선택 및 뉴스,토론방, 유튜브 데이터 통합

In [3]:
# LG화학, 삼성SDI, SK이노베이션, 고려아연, 포스코케미칼
stock_name = '삼성SDI'

In [4]:
naver_news = pd.read_csv('./data/refined_naver_news.csv', index_col = 0)
daum_news = pd.read_csv('./data/refined_daum_news.csv', index_col = 0)
naver_talks = pd.read_csv(f'./data/refined_naver_talks_{stock_name}.csv', index_col = 0)
daum_talks = pd.read_csv(f'./data/refined_daum_talks_{stock_name}.csv', index_col = 0)
youtube = pd.read_csv(f'./data/refined_youtube_{stock_name}.csv', index_col = 0)

In [5]:
# 데이터 통합
news_df = pd.concat([naver_news, daum_news, naver_talks, daum_talks ,youtube])

# 'Date' 타입이 int 이므로 datetime으로 변환
news_df['Date'] = pd.to_datetime(news_df['Date'].astype(str))

# 합쳐진 데이터들의 인덱스 재설정
news_df.sort_values('Date', ignore_index=True, inplace=True)

In [6]:
# 2021년 주식 데이터가 1월 4일부터 있어서 슬라이싱
news_df = news_df[news_df[news_df['Date']== '2021-01-04'].index[0] : ]
news_df.head(2)

,Date,Title
639,2021-01-04,속보 새해 첫 거래일 코스피 0 04 오른 2874 50 출발
640,2021-01-04,대웅제약 코로나 치료제 임상3상 시험 시작 주가 상승


# 주가 데이터

In [8]:
stock_df = pd.read_csv(f'./data/{stock_name}_주가_데이터.csv', usecols = ['일자','등락률'])
stock_df['일자'] = pd.to_datetime(stock_df['일자'])
stock_df.head(2)

,일자,등락률
0,2021-01-04,6.85
1,2021-01-05,2.24


In [9]:
start = str(stock_df.iloc[0, 0])
end = str(stock_df.iloc[-1, 0])
print(start)
print(end)

2021-01-04 00:00:00
2022-06-30 00:00:00


In [10]:
# multi 분류
stock_df['updown'] = 0

stock_df.loc[stock_df.query('등락률 > 1').index, 'updown'] = 1

stock_df.loc[stock_df.query('등락률 < -1').index, 'updown'] = -1
stock_df.head(2)

,일자,등락률,updown
0,2021-01-04,6.85,1
1,2021-01-05,2.24,1


# 데이터 프레임 합치기

In [11]:
## 뉴스일자 조정(예측대상(주가)의 일자와 맞추기 위해)
news_df['일자'] = news_df['Date'] + timedelta(days=1)

In [12]:
df = news_df.merge(stock_df)
df.columns = [df.columns[0], df.columns[1], '주가의 날짜', '등락률(y)', 'updown']
df.drop_duplicates('Title', inplace = True, ignore_index = True)  # 기사제목 중복 제거
print(len(df))
df.head()

431484


,Date,Title,주가의 날짜,등락률(y),updown
0,2021-01-04,속보 새해 첫 거래일 코스피 0 04 오른 2874 50 출발,2021-01-05,2.24,1
1,2021-01-04,대웅제약 코로나 치료제 임상3상 시험 시작 주가 상승,2021-01-05,2.24,1
2,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05,2.24,1
3,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결,2021-01-05,2.24,1
4,2021-01-04,코스닥 하락 전환 968 31 감소0 01,2021-01-05,2.24,1


# 감성사전 load

In [13]:
sentiment_csv = pd.read_csv('./sentiment dictionary.csv', index_col = 0)
sentiment_csv.head()

,pos,mid,neg
0,방긋,아직,회의
1,상회,보통,바닥
2,신선,vs,이탈
3,신박,중립,떨어지
4,투혼,관망,안좋게


In [14]:
pos_li = sentiment_csv['pos'].dropna().values
mid_li = sentiment_csv['mid'].dropna().values
neg_li = sentiment_csv['neg'].dropna().values

# 감성 지수 계산하는 sentimental_score() 

In [15]:
def sentimental_score(df):
    # 입력받은 데이터프레임 복사 및 컬럼 추가
    df_result = df.copy()
    df_result['Pos'] = 0
    df_result['Neg'] = 0
    df_result['Mid'] = 0
    
    # 감성 지수는 긍정 : 1, 중립 : 0, 부정 : -1, 해당 데이터 제외 : 999
    df_result['감성지수'] = 999    
    
    # 감성 사전에 따른 텍스트 검출
    print('긍정 단어 검색중')
    for pos in tqdm(pos_li) :
        str_expr = f"Title.str.contains('{pos}')"
        df_result.loc[df_result.query(str_expr).index, 'Pos'] = 1
    
    print('부정 단어 검색중')
    for neg in tqdm(neg_li) :
        str_expr = f"Title.str.contains('{neg}')"
        df_result.loc[df_result.query(str_expr).index, 'Neg'] = 1
    
    print('중립 단어 검색중')
    for mid in tqdm(mid_li) :
        str_expr = f"Title.str.contains('{mid}')"
        df_result.loc[df_result.query(str_expr).index, 'Mid'] = 1
    
    # 모든 종류의 단어가 검출 되면 제외
    df_result.loc[df_result.query('Pos == 1 and Neg == 1 and Mid == 1').index, '감성지수'] = 999
    
    # 중립 단어가 검출되면 중립
    df_result.loc[df_result.query('Mid == 1').index, '감성지수'] = 0
    
    # 긍정 단어만이 검출되면 긍정
    df_result.loc[df_result.query('Pos == 1 and Neg == 0 and Mid == 0').index, '감성지수'] = 1
    
    # 부정 단어만이 검출되면 부정
    df_result.loc[df_result.query('Pos == 0 and Neg == 1 and Mid == 0').index, '감성지수'] = -1
    
    
    # 긍정, 부정 단어가 둘 다 있으면 전 날 또는 당일 주가의 등락률을 보고 결정
    print('긍정 부정 둘 다 있는 경우 처리중')
    for i in tqdm(df_result.loc[df_result.query('Pos == 1 and Neg == 1 and Mid == 0').index].index) : 
        
        updown = 999 # 등락률을 뜻하는 updown
        
        # 해당 Title의 어제 주가가 있으면 선택
        if sum(df_result.loc[i,'Date'] - timedelta(days = 1) == stock_df['일자']) == 1 :  
            updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = 1)]['등락률'].values[0]
        
        # 어제 주가는 없지만 당일이 있으면 당일을 선택
        elif sum(df_result.loc[i,'Date'] == stock_df['일자']) == 1 :  
            updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date']]['등락률'].values[0]

        # 어제와 오늘의 주가도 없다면 이전의 주가를 찾아 탐색
        else :
            j = 2 
            while True :
                if sum(df_result.loc[i,'Date'] - timedelta(days = j) == stock_df['일자']) == 1 :
                    updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = j)]['등락률'].values[0]
                    break
                j += 1
        
        # 절댓값이 0보다 낮은 등락률은 변화가 없다고 판단
        if updown > 1 :
            df_result.loc[i,'감성지수'] = 1
        elif updown < 1 :
            df_result.loc[i,'감성지수'] = -1
        else :
            df_result.loc[i,'감성지수'] = 0

    return df_result

In [16]:
df_result = sentimental_score(df)

긍정 단어 검색중


100%|██████████| 328/328 [00:38<00:00,  8.44it/s]


부정 단어 검색중


100%|██████████| 295/295 [00:34<00:00,  8.60it/s]


중립 단어 검색중


100%|██████████| 11/11 [00:01<00:00,  8.57it/s]


긍정 부정 둘 다 있는 경우 처리중


100%|██████████| 72535/72535 [00:28<00:00, 2543.47it/s]


# 예측 모델 적용

In [26]:
# 형태소 분석을 위한 함수
def tokenizer(text):
    okt = Okt()
    return okt.morphs(text)

In [33]:
# X : title, y : price
def data_split(X, y):
    # 수집한 데이터 읽어오기
    
    # 학습셋, 테스트셋 분리
    X_list = X.tolist()
    y_list = y.tolist()
    
    X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, shuffle = False, test_size = 0.2)
    
    return X_train, X_test, y_train, y_test

In [34]:
X_train, X_test, y_train, y_test = data_split(df_result['Title'], df_result['감성지수'])

In [36]:
vect = CountVectorizer()

tfidf = TfidfTransformer()

In [37]:
#  멀티 로지스틱
logistic = LogisticRegression(multi_class = 'multinomial',  random_state=0)
# lbfgs의 경우 제약조건 L2만 지원.... 뭔소리지?

# logistic = LogisticRegression(multi_class = 'multinomial', solver='saga' random_state=0)
# saga는 L1, L2 지원, 확률적경사하강법 기반   multi_class = 'ovr'... 모지?

# logistic = LogisticRegression(C=2, penalty='l2', random_state=0)

# C의 숫자가 너무 크면 과적합 (기본 1), penalty로 과적합 방지

### 파이프라인

In [38]:
pipeline = Pipeline([('vect',vect), ('tfidf',tfidf), ('clf',logistic)], verbose=True)

In [40]:
pipeline.fit(X_train, y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total=   2.2s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.1s
[Pipeline] ............... (step 3 of 3) Processing clf, total=  21.8s


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 LogisticRegression(multi_class='multinomial',
                                    random_state=0))],
         verbose=True)

In [42]:
y_pred = pipeline.predict(X_test)

In [43]:
y_df = df['updown'][len(X_train) : ]

# 모델 평가

In [93]:
print(f'accuracy :\t {accuracy_score(y_test, y_pred)}')

#'micro', 'macro', 'weighted'
print(f'f1_score :\t {f1_score(y_test, y_pred, average = "micro")}')
print(f'f1_score :\t {f1_score(y_test, y_pred, average = "macro")}')
print(f'f1_score :\t {f1_score(y_test, y_pred, average = "weighted")}')

print(f'R2 \t:\t {r2_score(y_test, y_pred)}')

print(f'mse \t:\t {mean_squared_error(y_test, y_pred)}')

accuracy :	 0.8299013870702342
f1_score :	 0.8299013870702342
f1_score :	 0.8351385856313709
f1_score :	 0.8293308756003326
R2 	:	 0.5156571477198297
mse 	:	 76692.11309779019


In [95]:
scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring='f1_micro')
print('평균 모델 score:', scores.mean())

[Pipeline] .............. (step 1 of 3) Processing vect, total=   1.4s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.1s
[Pipeline] ............... (step 3 of 3) Processing clf, total=  15.0s
[Pipeline] .............. (step 1 of 3) Processing vect, total=   1.5s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.1s
[Pipeline] ............... (step 3 of 3) Processing clf, total=  15.4s
[Pipeline] .............. (step 1 of 3) Processing vect, total=   1.4s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.1s
[Pipeline] ............... (step 3 of 3) Processing clf, total=  16.5s
평균 모델 score: 0.842769293903713


# 모델 저장 및 사용

In [100]:
def save_model(model):
    with open('[Model5]pipe.dat', 'wb') as fp:     # 쓰기, 바탕화면에 저장됨
        pickle.dump(model, fp)
    print('저장완료')     # 학습된 모델 저장 완료

In [101]:
save_model(pipeline)

저장완료


In [105]:
# 모델 사용 함수
def model_prediction():  
    # 객체를 복원, 저장된 모델 불러오기
    with open('[Model5]pipe.dat','rb') as fp:     # 읽기
        pipe = pickle.load(fp)
    while True :
        text = input('뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : ')
        example = [text]
        # 예측 정확도
        r1 = np.max(pipe.predict_proba(example)*100)     # 확률값을 구해서 *100
        # 예측 결과
        r2 = pipe.predict(example)[0]     # 긍정('1'), 부정('0'), 중립('-1)
        if text == 'q':
            print("예측을 종료합니다.")
            break
        if r2 == '1':
            print(f'{stock_name} 주가가 상승할 것으로 예상됩니다.')
        elif r2 == '-1':
            print(f'{stock_name} 주가가 하락할 것으로 예상됩니다.')
        else: 
            print(f'모르겠어요.....')
        print('확률 : %.3f' % r1)
        print('------------------------------------------------')

In [108]:
model_prediction()

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) :  [줌인 이종목] 현대중공업, 실적 증가 기대에 껑충


모르겠어요.....
확률 : 99.410
------------------------------------------------


뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) :  닷새만에 반등했지만…불안한 개미들, 7100억 차익실현


모르겠어요.....
확률 : 78.626
------------------------------------------------


뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) :  주식거래 뚝…"증권사, 2분기 실적 쇼크"


모르겠어요.....
확률 : 93.423
------------------------------------------------


뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) :  q


예측을 종료합니다.
